# K-12 Diversity Statements Scraper

In [1]:
# Data manipulation libraries
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
# Common webscraping libaries
from bs4 import BeautifulSoup as bs
import requests

tqdm.pandas()

### Get list of universities

In [2]:
input_path = "../data/input/Private high schools.xlsx"
uni_list = pd.read_excel(input_path)
uni_list = uni_list.melt(id_vars=['School'], 
              value_vars=[x for x in uni_list.columns if "diversity" in x], 
              var_name='url_type', value_name='url')
uni_list = uni_list.dropna(subset=["url"])
print(uni_list.shape)
uni_list = uni_list.drop_duplicates(subset="url")
print(uni_list.shape)
uni_list

(431, 3)
(422, 3)


,School,url_type,url
0,Phillips Academy,diversity page 1,https://www.andover.edu/equity-inclusion-justice
1,Harvard-Westlake School,diversity page 1,https://www.hw.com/about/Diversity-Equity-Incl...
2,Phillips Exeter Academy,diversity page 1,https://www.exeter.edu/community/inclusive-com...
3,Trinity School,diversity page 1,https://www.trinityschoolnyc.org/about/diversity
4,Choate Rosemary Hall,diversity page 1,https://www.choate.edu/about/equity-and-inclusion
...,...,...,...
1504,Cincinnati Country Day School,diversity page 2,https://www.countryday.net/about-ccds/welcome/...
1552,St. George's School,diversity page 2,https://www.stgeorges.edu/guiding-principles
1686,The Walker School,diversity page 2,https://www.thewalkerschool.org/News-Detail?pk...
1711,Convent of the Visitation School,diversity page 2,https://www.visitation.net/uploaded/photos/New...


In [3]:
from utils import scrape_webpage
uni_list['text'] = uni_list.progress_apply(lambda x : scrape_webpage(x["url"],
                                                            x["School"],
                                                            x["url_type"],
                                                            out_folder="k-12_output",
                                                            save_images=False,
                                                            v=False),axis=1)
uni_list['failed_scrape'] = uni_list['text'].str.len() < 100
uni_list[['School','url_type','url','failed_scrape','text']]


  0%|          | 0/422 [00:00<?, ?it/s]

In [10]:
import re
uni_list = pd.read_csv("../data/k-12_output/k-12_diversity_statements.csv")
uni_list['text'] = uni_list['text'].map(lambda x : re.sub(r'[^ \w+]', '', x))
uni_list
# uni_list.to_csv(f"../data/k-12_output/pilot_websites.csv")

TypeError: expected string or bytes-like object